In [1]:
import pandas as pd

gen_df = pd.read_parquet("data/eia_hourly_generation_2025.parquet")

### Generation

Columns:
- period (in "%Y-%m-%dT%H")
- respondent
    - respondent code
- respondent-name
    - balancing authority
- fueltype
    - the kind of fuel
- type-name
    - long name of fuel type
- value
    - number of megawatt hours (MWh) generated
You can actually do a lot with this. might be interesting to understand where all of these regions are abd what fueltypes they are using. generation is going to be seasonal. 

In [3]:
gen_df.head()

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2024-12-31T00,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,1279,megawatthours
1,2024-12-31T00,AECI,"Associated Electric Cooperative, Inc.",NG,Natural Gas,1148,megawatthours
2,2024-12-31T00,AECI,"Associated Electric Cooperative, Inc.",WND,Wind,532,megawatthours
3,2024-12-31T00,AVA,Avista Corporation,NG,Natural Gas,225,megawatthours
4,2024-12-31T00,AVA,Avista Corporation,OTH,Other,117,megawatthours


We can fuzzy merge this with data in the excel spreadsheet. Or we can skip this first set and only look at the second, it is very detailed and intteresting, just lacks hourly observations. in The excel spreadsheet we have 6 pages:

### [EIA Form 923 Data](https://www.eia.gov/electricity/data/eia923/)

***WE ARE MISSING DATA IN THIS SET FOR DECEMBER***

1) Generation and Fuel Data
    - Output and fuel consumption by plant and by month
2) Stocks data
    - Page 2 Stocks Data 
        - Oil/coal stocks by census region and state
    - Page 2 Oil Stocks Data
        - Monthly oil stocks by plant and month
3) Boiler fuel data
    - relates to combined heat/power plants, what fuel they used (not particularly useful)
4) Generator  data
    - plants and how much energy they generated on a monthly basis
5) Fuel receipts and costs
    - really interesting, we can see what kinds of fuels plants purchased and how much + the cost, including details about where it came from and its contents
6) Plant frame
    - This describes whether the plant is producing electricity or heat + electricity (combined heat and power, or CHP)

#### Intuition

I think we can combine the first dataset, the hourly generation by region with the second one. Would be interesting to look at fuel receipts and costs, stocks data and gen + fuel data, and maybe we can predict the cost of electricity based on the fuel costs?

In [28]:
import re

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [
    "_".join([str(level) for level in col_tuple]).strip()
    for col_tuple in df.columns
    ]   
    df.columns = [re.sub(r'Unnamed:\s*\d+_?', '', col) for col in df.columns]
    df.columns = [re.sub(r'level_\d+', '', col) for col in df.columns]
    df.columns = [col.replace("__", "_").strip("_").strip() for col in df.columns]
    df.columns = [[col.replace("\n", " ") for col in df.columns]]
    return df

file_loc = "data/EIA923_Schedules_2_3_4_5_M_11_2024_21JAN2025.xlsx"
gen_data_agg = pd.read_excel(file_loc, sheet_name="Page 1 Generation and Fuel Data", skiprows=4, header=[0, 1])
gen_data_agg = clean_columns(gen_data_agg)

stocks_data_by_plant = pd.read_excel(file_loc, sheet_name="Page 2 Oil Stocks Data", skiprows=3, header=[0, 1])
stocks_data_by_plant = clean_columns(stocks_data_by_plant)

fuel_rct_by_plant = pd.read_excel(file_loc, sheet_name="Page 5 Fuel Receipts and Costs", skiprows=3)
fuel_rct_by_plant.columns = [col.replace("\n", " ") for col in fuel_rct_by_plant.columns]

In [22]:
gen_data_agg.head()

,Plant Id,Combined Heat And Power Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,...,Electricity Net Generation (MWh)_Netgen September,Electricity Net Generation (MWh)_Netgen October,Electricity Net Generation (MWh)_Netgen November,Electricity Net Generation (MWh)_Netgen December,Year-To-Date_Total Fuel Consumption Quantity,Year-To-Date_Electric Fuel Consumption Quantity,Year-To-Date_Total Fuel Consumption MMBtu,Year-To-Date_Elec Fuel Consumption MMBtu,Year-To-Date_Net Generation (Megawatthours),Year-To-Date_YEAR
0,3,N,.,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,400744,380868,346913,.,2991907,2991907,3054067,3054067,4214946.00,2024
1,3,N,.,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,762657,735257,675220,.,78666206,78666206,80306604,80306604,8040695.00,2024
2,3,N,.,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,20325.048,322168.46,11097.797,.,395622,395622,7810960,7810960,697528.56,2024
3,3,N,.,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,3238.952,6842.536,731.203,.,2375291,2375291,2426415,2426415,174407.44,2024
4,3,N,.,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,0,0,0,.,0,0,0,0,0.00,2024


In [24]:
stocks_data_by_plant.head()

,Plant Id,Combined Heat And Power Plant,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,NAICS Code,EIA Sector Number,...,Total Month Ending Stocks In Physical Units)_Quantity May,Total Month Ending Stocks In Physical Units)_Quantity June,Total Month Ending Stocks In Physical Units)_Quantity July,Total Month Ending Stocks In Physical Units)_Quantity August,Total Month Ending Stocks In Physical Units)_Quantity September,Total Month Ending Stocks In Physical Units)_Quantity October,Total Month Ending Stocks In Physical Units)_Quantity November,Total Month Ending Stocks In Physical Units)_Quantity December,Total Month Ending Stocks In Physical Units)_Balancing Authority Code,Total Month Ending Stocks In Physical Units)_YEAR
0,10,N,Greene County,Alabama Power Co,195,AL,ESC,SERC,22,1,...,126862,126813,126612,126372,126051,126025,125973,.,SOCO,2024
1,26,N,E C Gaston,Alabama Power Co,195,AL,ESC,SERC,22,1,...,2078,2034,887,880,359,1868,1795,.,SOCO,2024
2,54,N,J K Smith,"East Kentucky Power Coop, Inc",5580,KY,ESC,SERC,22,1,...,84788,84783,84777,84772,84724,84315,84157,.,PJM,2024
3,60,N,Whelan Energy Center,City of Hastings - (NE),8245,NE,WNC,MRO,22,1,...,2298,1658,1470,1370,1302,933,1923,.,SWPP,2024
4,113,N,Cholla,Arizona Public Service Co,803,AZ,MTN,WECC,22,1,...,3524,0,25332,0,0,24852,24852,.,AZPS,2024


In [26]:
fuel_rct_by_plant.head()

,YEAR,MONTH,Plant Id,Plant Name,Plant State,Purchase Type,Contract\nExpiration Date,ENERGY_SOURCE,FUEL_GROUP,Coalmine\nType,...,Regulated,Operator Name,Operator Id,Primary Transportation Mode,Secondary Transportation Mode,Natural Gas Supply Contract Type,Natural Gas Delivery Contract Type,Moisture\nContent,Chlorine\nContent,BA_CODE
0,2024,1,3,Barry,AL,C,624,BIT,Coal,S,...,REG,Alabama Power Co,195,RV,NaN,NaN,NaN,16,0,SOCO
1,2024,1,3,Barry,AL,S,.,NG,Natural Gas,NaN,...,REG,Alabama Power Co,195,PL,NaN,F,F,.,.,SOCO
2,2024,1,3,Barry,AL,S,.,NG,Natural Gas,NaN,...,REG,Alabama Power Co,195,PL,NaN,F,F,.,.,SOCO
3,2024,1,10,Greene County,AL,S,.,NG,Natural Gas,NaN,...,REG,Alabama Power Co,195,PL,NaN,F,F,.,.,SOCO
4,2024,1,26,E C Gaston,AL,C,1224,BIT,Coal,S,...,REG,Alabama Power Co,195,RR,NaN,NaN,NaN,6.3,0,SOCO
